In [130]:
import numpy as np
import sympy as sp
from typing import List
from sympy.plotting import plot
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Math
def eq_disp(varstring, expr):
    display(Latex(f"${varstring}={sp.latex(expr)}$"))

def reduce_feedback(G_fwd, G_bwd):
    """Assumes feedback is deducted from signal, if not
    change sign of feedback"""
    return sp.simplify(G_fwd/(1+G_fwd*G_bwd))

def RHarray(coeffs: List):
    # first 2 rows from coefficients
    n = len(coeffs)
    arr = sp.zeros(n, n//2+2)
    i = 0
    for i in range(0,n,2):
        arr[0, i//2] = coeffs[i]
    for i in range(1,n,2):
        arr[1, i//2] = coeffs[i]

    for j in range(2, arr.shape[0]):
        for i in range(arr.shape[1]-1):
            a0 = arr[j-2,0]
            a3 = a1 = arr[j-1,i+1]
            a1 = arr[j-1,0]
            a2 = arr[j-2,i+1]
            arr[j, i] = (a1*a2-a0*a3)/a1
    return arr



In [131]:
K1, K2, K3, s, t = sp.symbols('K1, K2, K3, s, t')
tau1, tau3, tau4 = 0.5, 1, 1/4
Gs = K1/(tau1*s + 1)
Gd = K3/((tau3*s + 1)*(tau4*s + 1))

In [132]:
T = reduce_feedback(Gs*K2*Gd, 1)
T

K1*K2*K3/(K1*K2*K3 + (0.25*s + 1)*(0.5*s + 1)*(s + 1))

The coefficients of the characteristic equation are

In [133]:
p, q = T.as_numer_denom()
coeffs = sp.Poly(q, s).coeffs()
for i, k in enumerate(coeffs):
    display(Latex(f"${f's^{len(coeffs)-1-i}'}: {sp.latex(k)}$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [134]:
arr = RHarray(coeffs)
arr

Matrix([
[                                         0.125,               1.75, 0, 0],
[                                         0.875, 1.0*K1*K2*K3 + 1.0, 0, 0],
[-0.142857142857143*K1*K2*K3 + 1.60714285714286,                  0, 0, 0],
[                            1.0*K1*K2*K3 + 1.0,                  0, 0, 0]])

Now applying the Routh Hurwitz criterion to the first columnm, all coefficients should be positive, which gives the bounding equations:

In [135]:
K = sp.symbols('K')
sol = sp.solve([arr[2, 0].subs(K1*K2*K3, K) > 0, arr[3, 0].subs(K1*K2*K3, K) > 0], K).subs(K, K1*K2*K3)
sol

(-1.0 < K1*K2*K3) & (K1*K2*K3 < 11.25)